In [ ]:
#for dev purpose
import getpass
import os
USERNAME = os.getenv('USERNAME').lower()
new_pull = True

In [ ]:
# import python packages
import os
import sys
if sys.version_info[1]==12:
    os.add_dll_directory(r'C:/Program Files/Python/312/Lib/site-packages/clidriver/bin')
else:
    pass
import pandas as pd
import datetime as dt
import time
import warnings
from snp_query_box import DsnpHelperFunction, snpQueries
from snp_query_box.dsnp_transform import memberContactLegalTransform
warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)
print(f"User: {USERNAME} Started creating mbr_contact_legal_short_valid_df in the shared storage")

In [ ]:
# set the starting time
start_time = time.time()

In [ ]:
# calculate the dates
today = dt.datetime.today()
today_str = today.strftime('%Y-%m-%d')
last_date_of_last_month_str = DsnpHelperFunction.last_date_of_last_month(today_str, output_type="string")
first_date_of_last_month = DsnpHelperFunction.first_date_of_last_month(today_str)
last_day_reporting_year = DsnpHelperFunction.last_date_of_year(first_date_of_last_month)
reporting_end_date = last_date_of_last_month_str
print(reporting_end_date)

In [ ]:
# set the storage and output path
storage_path = r'//mbip/medicarepBI/Projects/COE/DSNP/dsnp_data_storage/monthly_flat_files'
output_path = f'{storage_path}/{reporting_end_date}'

isExist = os.path.exists(output_path)
print(output_path)
if not isExist:
    # Create a new directory because it does not exist
     os.makedirs(output_path)
     print("The new directory is created!")
else:
      print("The folder already exist")
print("Output files will sit in the folder above.")

In [ ]:
#locate data dump folder
if new_pull:
    pull_member_contact_legal = snpQueries.pull_mbr_contact_legal_bq()
    pull_member_contact_legal.to_parquet(f"{output_path}/pull_member_contact_legal.parquet")
else:
    try:
        pull_member_contact_legal = pd.read_parquet(f"{output_path}/pull_member_contact_legal.parquet")

    except:
        pull_member_contact_legal = snpQueries.pull_mbr_contact_legal_bq()
        pull_member_contact_legal.to_parquet(f"{output_path}/pull_member_contact_legal.parquet")

In [ ]:
mbr_contact_legal_short_valid_df = memberContactLegalTransform.member_contact_legal_transform(pull_member_contact_legal, first_date_of_last_month, last_day_reporting_year)
mbr_contact_legal_short_valid_df.to_parquet(f"{output_path}/mbr_contact_legal_short_valid_df.parquet")

In [ ]:
# output the top rows and size

print(mbr_contact_legal_short_valid_df.shape)
print(mbr_contact_legal_short_valid_df.head(3))

In [ ]:
# output the process time
print("Process time -- %s seconds" % (time.time() - start_time))